In [ ]:
import os
from diffaux.disk_bulge_modeling.generate_bulge_disk_sample import (
    get_bulge_disk_test_sample,
    get_bulge_disk_decomposition,
)
from jax import random as jran
import numpy as np
import jax.numpy as jnp

ran_key = jran.key(0)

### Generate subcat and SFH catalog

In [ ]:
halo_key, ran_key = jran.split(ran_key, 2)
lgmp_min = 11.0
redshift = 0.05
Lbox = 100.0
diffstar_cens = get_bulge_disk_test_sample(halo_key, lgmp_min=lgmp_min, redshift=redshift, Lbox=Lbox)

### Explore Existing Model for Disk-Bulge Decomposition with logsm0 Bug-fixed version

In [ ]:
disk_bulge_key, ran_key = jran.split(ran_key, 2)
diffstar_cens = get_bulge_disk_decomposition(disk_bulge_key, diffstar_cens, new_model=False)
print(diffstar_cens.keys())

In [ ]:
zvalues = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
dz = 0.1
redshifts = diffstar_cens["z_table"]
from diffaux.validation.plot_disk_bulge import (
    plot_qs_nocuts,
    plot_histories,
    plot_q_with_cuts,
    plot_q1_q2,
    plot_q1_vs_q2,
)

plotdir = "/Users/kovacs/cosmology/BulgeDisk/DiskBulgePlots"
from diffstar.utils import cumulative_mstar_formed_galpop

smh = cumulative_mstar_formed_galpop(diffstar_cens["t_table"], diffstar_cens["sfh"])

In [ ]:
# plot smf
mass_bins = np.linspace(8.0, 11.6, 19)
print(mass_bins)
plot_qs_nocuts(
    [
        jnp.log10(diffstar_cens["smh_disk"]),
        jnp.log10(diffstar_cens["smh_bulge"]),
        jnp.log10(diffstar_cens["smh"]),
    ],
    zvalues,
    redshifts,
    dz=0.1,
    bins=mass_bins,
    qlabels=["Disk", "Bulge", "Total"],
    yscale="log",
    xscale="linear",
    plotdir=plotdir,
)

In [ ]:
qs = [
    diffstar_cens["eff_bulge"],
    diffstar_cens["smh_bulge"],
    diffstar_cens["smh"],
    diffstar_cens["bth"],
    # sfh_bulge, diffstar_cens['sfh'],
    diffstar_cens["sSFR_bulge"],
    diffstar_cens["sSFR"],
]
ylabels = [
    "Bulge_efficiency",
    "Bulge_SMH ($M_\\odot$)",
    "SMH ($M_\\odot$)",
    "B/T",
    #'Bulge_SFH ($M_\\odot \\mathrm{yr}^{-1}$)', 'SFH ($M_\\odot \\mathrm{yr}^{-1}$)',
    "Bulge_sSFH ($\\mathrm{yr}^{-1}$)",
    "sSFH ($\\mathrm{yr}^{-1}$)",
]
labels = [
    "effB",
    "SMHB",
    "SMH",
    "BT",
    #'SFHB', 'SFH',
    "sSFRB",
    "sSFR",
]

In [ ]:
logM_min = 8.0
row_mask = np.log10(diffstar_cens["smh"][:, -1]) > logM_min
print(np.count_nonzero(row_mask))
plot_histories(
    qs,
    diffstar_cens["t_table"],
    labels,
    ylabels,
    plotdir=plotdir,
    color_array=np.log10(diffstar_cens["smh"][:, -1]),
    row_mask=row_mask,
    yscale="log",
    plot_label="Mstar_z0",
    step=500,
    lgnd_label="$\\log_{{10}}(M^*/M_\\odot) = {:.1f}$",
)

In [ ]:
# setup mass mask
logMz0_min = 8.0
mass_mask = np.log10(diffstar_cens["smh"][:, -1]) > logMz0_min
lgnd_title = ", $\\log_{{10}}(M^*_{{z=0}}/M_\\odot) > {:.1f}$".format(logMz0_min)

plotdir = "/Users/kovacs/cosmology/BulgeDisk/DiskBulgePlots"

In [ ]:
plot_q_with_cuts(
    diffstar_cens["bth"][mass_mask],
    zvalues,
    redshifts,
    jnp.log10(diffstar_cens["sSFR"])[mass_mask],
    [-11, -10],
    dz=0.2,
    lgnd_title=lgnd_title,
    plotdir=plotdir,
)

In [ ]:
plot_q_with_cuts(
    diffstar_cens["eff_bulge"][mass_mask],
    zvalues,
    redshifts,
    np.log10(diffstar_cens["sSFR"])[mass_mask],
    [-11, -10],
    dz=0.2,
    pltname="effB_cut_on_{}.png",
    xlabel="Bulge Efficiency",
    lgnd_title=lgnd_title,
    plotdir=plotdir,
)

In [ ]:
logMz0_min = 8.5
mass_mask = np.log10(diffstar_cens["smh"][:, -1]) > logMz0_min
print(mass_mask.shape)
bins = np.logspace(7, 12, 51)
lgnd_title = ", $\\log_{{10}}(M^*_{{z=0}}/M_\\odot) > {:.1f}$".format(logMz0_min)
# print(lgnd_title)

plot_q_with_cuts(
    diffstar_cens["smh_bulge"][mass_mask],
    zvalues,
    redshifts,
    np.log10(diffstar_cens["sSFR"])[mass_mask],
    [-11, -10],
    dz=0.2,
    pltname="bulge_mass_cut_on_{}.png",
    xlabel="Bulge Mass ($M_\\odot$)",
    xscale="log",
    bins=bins,
    lgnd_title=lgnd_title,
    plotdir=plotdir,
)

In [ ]:
# logMz0_min = 8.5
# mass_mask = np.log10(diffstar_cens['smh'][:, -1]) > logMz0_min
bins = np.logspace(-14, -7, 71)
mass_bins = np.linspace(7.5, 11.5, 5)
print(mass_bins, np.min(bins), np.max(bins))
# test masking
zmask = redshifts < 0.2
q1_z = diffstar_cens["sSFR_bulge"][:, zmask]
cut_array_z = np.log10(diffstar_cens["smh"])[:, zmask]
cut_mask = (cut_array_z[:, -1] >= 8.5) & (cut_array_z[:, -1] < 11.5)
print(cut_mask.shape, np.where(cut_mask == False)[0][0:2])
cut_mask = np.broadcast_to(cut_mask, (np.count_nonzero(zmask), len(cut_mask))).T
print(cut_mask.shape)

In [ ]:
for m_lo, m_hi in zip(mass_bins[0:-1], mass_bins[1:]):
    plot_q1_q2(
        diffstar_cens["sSFR_bulge"],
        diffstar_cens["sSFR_disk"],
        zvalues,
        redshifts,
        np.log10(diffstar_cens["smh"]),
        m_lo,
        m_hi,
        dz=0.2,
        xname="log_M0_{:.1f}_{:.1f}",
        cut_name="$\\log_{10}(M^*_{z=0}/M_\\odot)$",
        cut_at_z0=True,
        bins=bins,
        plotdir=plotdir,
    )

In [ ]:
logMz0_min = 8.5
logssfr_min = -14
mass_mask = np.log10(diffstar_cens["smh"][:, -1]) > logMz0_min
title = "$M^*_{{z=0}} > 10^{{{:.1f}}} M_\\odot, sSFR > 10^{{{}}} yr^{{-1}}$".format(logMz0_min, logssfr_min)
xname = "log_M0_min_{:.1f}_logssfr_min_{}".format(logMz0_min, -logssfr_min)
plot_q1_vs_q2(
    jnp.log10(diffstar_cens["smh"][mass_mask]),
    jnp.log10(diffstar_cens["sSFR"][mass_mask]),
    zvalues,
    redshifts,
    diffstar_cens["bth"][mass_mask],
    title=title,
    xname=xname,
    ymin=logssfr_min,
    N=2000,
    plotdir=plotdir,
)

In [ ]:
logMz0_min = 8.5
logssfr_min = -14
mass_mask = np.log10(diffstar_cens["smh"][:, -1]) > logMz0_min
title = "$M^*_{{z=0}} > 10^{{{:.1f}}} M_\\odot, sSFR > 10^{{{}}} yr^{{-1}}$".format(logMz0_min, logssfr_min)
xname = "log_M0_min_{:.1f}_logssfr_min_{}".format(logMz0_min, -logssfr_min)
plot_q1_vs_q2(
    jnp.log10(diffstar_cens["smh_bulge"][mass_mask]),
    jnp.log10(diffstar_cens["sSFR"][mass_mask]),
    zvalues,
    redshifts,
    diffstar_cens["bth"][mass_mask],
    title=title,
    xname=xname,
    ymin=logssfr_min,
    N=2000,
    plotdir=plotdir,
    xlabel="$\\log_{10}(M^*_{bulge}/M_\\odot)$",
    pltname="sSFR_vs_Mbulge_{}.png",
)